<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/managed/vectaraDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vectara Managed Index
In this notebook we are going to show how to use [Vectara](https://vectara.com) with LlamaIndex.
Vectara is the first example of a "Managed" Index, a new type of index in Llama-index which is managed via an API.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
!pip install llama-index

In [ ]:
from llama_index import SimpleDirectoryReader
from llama_index.indices import VectaraIndex

### Loading documents
Load the documents stored in the `Uber 10q` using the SimpleDirectoryReader

In [ ]:
documents = SimpleDirectoryReader(os.path.abspath("../data/10q/")).load_data()
print(f"documents loaded into {len(documents)} document objects")
print(f"Document ID of first doc is {documents[0].doc_id}")

documents loaded into 305 document objects
Document ID of first doc is f5956977-ab53-468b-a38e-080def7866e4


### Add the content of the documents into a pre-created Vectara corpus
Here we assume an empty corpus is created and the details are available as environment variables:
* VECTARA_CORPUS_ID
* VECTARA_CUSTOMER_ID
* VECTARA_API_KEY

In [ ]:
index = VectaraIndex.from_documents(documents)

### Query the Vectara Index
We can now ask questions using the VectaraIndex retriever.

In [ ]:
query = "Is Uber still losing money or have they achieved profitability?"

First we use the retriever to list the returned documents:

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=5)
response = query_engine.retrieve(query)
texts = [t.node.text for t in response]
print("\n--\n".join(texts))

Most jurisdictions in which we operate have laws that govern payment and financial services activities. Regulators in certain jurisdictions may determine that
certain aspects of our business are subject to these laws and could require us to obtain licenses to continue to operate in such jurisdictions. For example, our
subsidiary in the Netherlands, Uber Payments B.V., is registered and authorized by its competent authority, De Nederlandsche Bank, as an electronic money
institution. This authorization permits Uber Payments B.V. to provide payment services (including acquiring and executing payment transactions and money
remittances, as referred to in the Revised Payment Services Directive (2015/2366/EU)) and to issue electronic money in the Netherlands. In addition, Uber
Payments B.V. has notified De Nederlandsche Bank that it will provide such services on a cross-border passport basis into other countries within the EEA.
--
Most jurisdictions in which we operate have laws that govern p

with the as_query_engine(), we can ask questions and get the responses based on Vectara's full RAG pipeline:

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=5)
response = query_engine.query(query)
print(response)

As of the provided search results, there is no direct information about Uber's current financial state or whether they have achieved profitability. However, the search results mention that Uber is facing regulatory challenges in different jurisdictions [1][3]. These challenges involve the classification of drivers as employees and social security contributions [3]. The outcome of these cases could affect Uber's financial situation. It is important to note that the search results did not provide a clear answer regarding whether Uber is still losing money or if they have achieved profitability.


Note that the "response" object above includes both the summary text but also the source documents used to provide this response (citations)

Vectara supports max-marginal-relevance natively in the backend, and this is available as a query mode. 
Let's see an example of how to use MMR: We will run the same query "Is Uber still losing money or have they achieved profitability?" but this time we will use MMR where mmr_diversity_bias=1.0 which maximizes the focus on maximum diversity:

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    n_sentences_before=2,
    n_sentences_after=2,
    vectara_query_mode="mmr",
    mmr_k=50,
    mmr_diversity_bias=1.0,
)
response = query_engine.retrieve(query)

texts = [t.node.text for t in response]
print("\n--\n".join(texts))

We are challenging each of them before the Social Security and Administrative Tribunals. In April 2021, a ruling was made that Uber Switzerland could not be held liable for social security contributions. The litigations with regards to Uber B.V. and
Rasier Operations B.V. are still pending for years 2014 to 2019. In January 2022, the Social Security Tribunal of Zurich reclassified drivers who have used the App
in 2014 as dependent workers of Uber B.V. and Rasier Operations B.V. from a social security standpoint, but this ruling has been appealed before the Federal
Tribunal and has no impact on our current operations. On June 3, 2022, the Federal Tribunal issued two rulings by which both Drivers and Couriers in the canton of
Geneva are classified as employees of Uber B.V. and Uber Switzerland GmbH.
--
If the requirement is not repealed or modified, our financial condition, operating results, and cash flows may be adversely impacted
by this legislation. In August 2022, the Inflation Redu

As you can see, the results in this case are much more diverse, and for example do not contain the same text more than once. The response is also better since the LLM had a more diverse set of facts to ground its response on:

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    n_sentences_before=2,
    n_sentences_after=2,
    summary_enabled=True,
    vectara_query_mode="mmr",
    mmr_k=50,
    mmr_diversity_bias=1.0,
)
response = query_engine.query(query)
print(response)

Based on the search results, the profitability of Uber is still uncertain. There are ongoing litigations and regulatory challenges in various jurisdictions regarding labor classification, social security contributions, and tax matters [1][3][4][6]. While Uber has reported revenue growth and improved adjusted EBITDA, it also incurred net losses due to factors such as unrealized losses on investments and stock-based compensation expenses [5]. The outcome of these legal and regulatory issues may impact Uber's financial condition and future profitability [1][3]. Therefore, it cannot be definitively stated whether Uber has achieved profitability or is still losing money.


So far we've used Vectara's internal summarization capability, which is the best way for most users.

You can still use Llama-Index's standard VectorStore as_query_engine() method, in which case Vectara's summarization won't be used, and you would be using an external LLM (like OpenAI's GPT-4 or similar) and a cutom prompt from LlamaIndex to generate the summart. For this option just set summary_enabled=False

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    summary_enabled=False,
    vectara_query_mode="mmr",
    mmr_k=50,
    mmr_diversity_bias=0.5,
)
response = query_engine.query(query)
print(response)

Uber is still losing money and has not achieved profitability.
